# Gradient Descent
Trong phần này, chúng ta sẽ giới thiệu các khái niệm cơ bản liên quan đến hạ gradient. Mặc dù phương pháp này hiếm khi được sử dụng trực tiếp trong học sâu, việc hiểu về hạ gradient là chìa khóa để hiểu các thuật toán hạ gradient ngẫu nhiên. Ví dụ, vấn đề tối ưu hóa có thể bị phân kỳ do tốc độ học quá lớn. Hiện tượng này đã có thể thấy trong hạ gradient. Tương tự, việc tiền điều kiện hóa là một kỹ thuật phổ biến trong hạ gradient và được áp dụng trong các thuật toán nâng cao hơn. Hãy bắt đầu với một trường hợp đặc biệt đơn giản.

## One-Dimensional Gradient Descent
Hạ gradient trong một chiều là một ví dụ tuyệt vời để giải thích tại sao thuật toán hạ gradient có thể giảm giá trị của hàm mục tiêu. Xét một hàm thực khả vi liên tục $f: \mathbb{R} \rightarrow \mathbb{R}$. Sử dụng khai triển Taylor, ta được:
$$f(x+\epsilon)=f(x)+\epsilon f^{\prime}(x)+O\left(\epsilon^2\right).$$
Tức là, trong xấp xỉ bậc một, $f(x+\epsilon)$ được xác định bởi giá trị hàm $f(x)$ và đạo hàm bậc một $f^{\prime}(x)$ tại $x$. Không vô lý khi giả định rằng với $\epsilon$ nhỏ, việc di chuyển theo hướng gradient âm sẽ giảm $f$. Để đơn giản, ta chọn một kích thước bước cố định $\eta>0$ và chọn $\epsilon=-\eta f^{\prime}(x)$. Thay vào khai triển Taylor ở trên, ta được:
$$f\left(x-\eta f^{\prime}(x)\right)=f(x)-\eta f^{\prime 2}(x)+O\left(\eta^2 f^{\prime 2}(x)\right).$$
Nếu đạo hàm $f^{\prime}(x) \neq 0$ không biến mất, ta đạt được tiến bộ vì $\eta f^{\prime 2}(x)>0$. Hơn nữa, ta luôn có thể chọn $\eta$ đủ nhỏ để các hạng bậc cao trở nên không đáng kể. Do đó, ta có:
$$f\left(x-\eta f^{\prime}(x)\right) \leq f(x).$$
Điều này có nghĩa là, nếu ta sử dụng:
$$x \leftarrow x-\eta f^{\prime}(x)$$
để lặp $x$, giá trị của hàm $f(x)$ có thể giảm. Do đó, trong hạ gradient, ta đầu tiên chọn một giá trị ban đầu $x$ và một hằng số $\eta>0$, sau đó sử dụng chúng để lặp $x$ liên tục cho đến khi đạt điều kiện dừng, ví dụ, khi độ lớn của gradient $\left|f^{\prime}(x)\right|$ đủ nhỏ hoặc số lần lặp đạt một giá trị nhất định.
Để đơn giản, ta chọn hàm mục tiêu $f(x)=x^2$ để minh họa cách thực hiện hạ gradient. Mặc dù ta biết rằng $x=0$ là nghiệm tối ưu của $f(x)$, ta vẫn sử dụng hàm đơn giản này để quan sát cách $x$ thay đổi.

In [ ]:
%matplotlib inline
import numpy as np
import torch
from d21 import torch as d21

def f(x):  # objective function
    return x ** 2

def f_grad(x):  # Gradient (derivative) of the objective function
    return 2 * x

Tiếp theo, ta sử dụng $x=10$ làm giá trị ban đầu và giả sử $\eta=0.2$. Sử dụng hạ gradient để lặp $x$ 10 lần, ta có thể thấy rằng cuối cùng, giá trị của $x$ tiến gần đến nghiệm tối ưu.

In [ ]:
def gd(eta, f_grad):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * f_grad(x)
        results.append(float(x))
    print(f'epoch 10, x: {x:f}')
    return results

results = gd(0.2, f_grad)

epoch 10, x: 0.060466

Tiến trình tối ưu hóa $x$ có thể được vẽ như sau.

In [ ]:
def show_trace(results, f):
    n = max(abs(min(results)), abs(max(results)))
    f_line = torch.arange(-n, n, 0.01)
    d21.set_figsize()
    d21.plot([f_line, results], [[f(x) for x in f_line], [f(x) for x in results]], 'x', 'f(x)', fmts=['-', '-o'])
    show_trace(results, f)

### Learning Rate
Tốc độ học (learning rate) 𝜂 có thể được thiết lập bởi người thiết kế thuật toán. Nếu chúng ta sử dụng một tốc độ học quá nhỏ, nó sẽ khiến `𝑥` cập nhật rất chậm, đòi hỏi nhiều vòng lặp hơn để thu được nghiệm tốt hơn. Để minh họa điều xảy ra trong trường hợp như vậy, hãy xem xét tiến trình trong cùng bài toán tối ưu với 𝜂 = 0.05. Như ta có thể thấy, ngay cả sau 10 bước lặp, chúng ta vẫn còn cách rất xa nghiệm tối ưu.

In [ ]:
show_trace(gd(0.05, f_grad), f)

epoch 10, x: 3.486784

Ngược lại, nếu chúng ta sử dụng tốc độ học quá lớn, giá trị `|𝜂 𝑓'(𝑥)|` có thể trở nên quá lớn đối với công thức khai triển Taylor bậc nhất. Nghĩa là, số hạng `O (𝜂² 𝑓'²(𝑥))` trong công thức (12.3.2) có thể trở nên đáng kể. Trong trường hợp này, chúng ta không thể đảm bảo rằng quá trình cập nhật lặp của `𝑥` sẽ làm giảm giá trị của hàm `𝑓(𝑥)`. Ví dụ, khi chúng ta đặt tốc độ học `𝜂 = 1.1`, `𝑥` vượt quá (overshoots) nghiệm tối ưu `𝑥 = 0` và dần dần phân kỳ.

In [ ]:
show_trace(gd(1.1, f_grad), f)

epoch 10, x: 61.917364

Để minh họa điều gì xảy ra với các hàm không lồi, hãy xem xét trường hợp $f(x)=x \cdot \cos(cx)$ với một hằng số $c$. Hàm này có vô số cực tiểu cục bộ. Tùy thuộc vào lựa chọn tốc độ học và mức độ điều kiện của bài toán, ta có thể đạt được một trong nhiều nghiệm. Ví dụ dưới đây minh họa cách một tốc độ học cao (không thực tế) sẽ dẫn đến một cực tiểu cục bộ kém.

In [ ]:
c = torch.tensor(0.15 * np.pi)

def f(x):  # Objective function
    return x * torch.cos(c * x)

def f_grad(x):  # Gradient of the objective function
    return torch.cos(c * x) - c * x * torch.sin(c * x)

show_trace(gd(2, f_grad), f)

epoch 10, x: -1.528166

## Multivariate Gradient Descent
Bây giờ khi đã có trực giác tốt hơn về trường hợp một biến, hãy xem xét tình huống mà $\mathbf{x}=\left[x_1, x_2, \ldots, x_d\right]^{\top}$. Tức là, hàm mục tiêu $f: \mathbb{R}^d \rightarrow \mathbb{R}$ ánh xạ các vector thành số thực. Gradient của nó cũng là đa biến, là một vector gồm $d$ đạo hàm riêng:
$$\nabla f(\mathbf{x})=\left[\frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_d}\right]^{\top}.$$
Mỗi phần tử đạo hàm riêng $\partial f(\mathbf{x}) / \partial x_i$ trong gradient biểu thị tốc độ thay đổi của $f$ tại $\mathbf{x}$ theo đầu vào $x_i$. Như trong trường hợp một biến, ta có thể sử dụng xấp xỉ Taylor đa biến để có ý tưởng về việc nên làm gì. Cụ thể, ta có:
$$f(\mathbf{x}+\boldsymbol{\epsilon})=f(\mathbf{x})+\boldsymbol{\epsilon}^{\top} \nabla f(\mathbf{x})+O\left(|\boldsymbol{\epsilon}|^2\right).$$
Nói cách khác, đến các hạng bậc hai trong $\epsilon$, hướng giảm nhanh nhất được cho bởi gradient âm $-\nabla f(\mathbf{x})$. Chọn một tốc độ học phù hợp $\eta>0$ cho ra thuật toán hạ gradient nguyên mẫu:
$$\mathbf{x} \leftarrow \mathbf{x}-\eta \nabla f(\mathbf{x}).$$
Để thấy thuật toán hoạt động thế nào trong thực tế, hãy xây dựng một hàm mục tiêu $f(\mathbf{x})=x_1^2+2x_2^2$ với vector hai chiều $\mathbf{x}=\left[x_1, x_2\right]^{\top}$ làm đầu vào và một số thực làm đầu ra. Gradient được cho bởi $\nabla f(\mathbf{x})=\left[2x_1, 4x_2\right]^{\top}$. Ta sẽ quan sát quỹ đạo của $\mathbf{x}$ bằng hạ gradient từ vị trí ban đầu $[-5, -2]$.
Để bắt đầu, ta cần hai hàm hỗ trợ. Hàm đầu tiên sử dụng một hàm cập nhật và áp dụng nó 20 lần cho giá trị ban đầu. Hàm hỗ trợ thứ hai trực quan hóa quỹ đạo của $\mathbf{x}$.

In [ ]:
def train_2d(trainer, steps=20, f_grad=None):  # save
    """Optimize a 2D objective function with a customized trainer."""
    # s1 and s2 are internal state variables that will be used in Momentum,
    # Adagrad, RMSProp
    x1, x2, s1, s2 = -5, -2, 0, 0
    results = [(x1, x2)]
    for i in range(steps):
        if f_grad:
            x1, x2, s1, s2 = trainer(x1, x2, s1, s2, f_grad)
        else:
            x1, x2, s1, s2 = trainer(x1, x2, s1, s2)
        results.append((x1, x2))
    print(f'epoch {i + 1}, x1: {float(x1):f}, x2: {float(x2):f}')
    return results

def show_trace_2d(f, results):  # save
    """Show the trace of 2D variables during optimization."""
    d21.set_figsize()
    d21.plt.plot(*zip(*results), '-o', color='#ff7f0e')
    x1, x2 = torch.meshgrid(torch.arange(-5.5, 1.0, 0.1),
                            torch.arange(-3.0, 1.0, 0.1), indexing='ij')
    d21.plt.contour(x1, x2, f(x1, x2), colors='#1f77b4')
    d21.plt.xlabel('x1')
    d21.plt.ylabel('x2')

Tiếp theo, ta quan sát quỹ đạo của biến tối ưu $\mathbf{x}$ với tốc độ học $\eta=0.1$. Ta thấy rằng sau 20 bước, giá trị của $\mathbf{x}$ tiến gần đến cực tiểu tại $[0, 0]$. Tiến trình khá ổn định mặc dù khá chậm.

In [ ]:
def f_2d(x1, x2):  # Objective function
    return x1 ** 2 + 2 * x2 ** 2

def f_2d_grad(x1, x2):  # Gradient of the objective function
    return (2 * x1, 4 * x2)

def gd_2d(x1, x2, s1, s2, f_grad):
    g1, g2 = f_grad(x1, x2)
    return (x1 - eta * g1, x2 - eta * g2, 0, 0)

eta = 0.1
show_trace_2d(f_2d, train_2d(gd_2d, f_grad=f_2d_grad))

epoch 20, x1: -0.057646, x2: -0.000073

## Adaptive Methods
Như đã thấy trong Phần 12.3.1, việc chọn tốc độ học $\eta$ "vừa đúng" là một việc khó khăn. Nếu chọn quá nhỏ, ta tiến bộ rất ít. Nếu chọn quá lớn, nghiệm sẽ dao động và trong trường hợp xấu nhất, có thể phân kỳ. Điều gì sẽ xảy ra nếu ta có thể tự động xác định $\eta$ hoặc loại bỏ hoàn toàn việc chọn tốc độ học? Các phương pháp bậc hai, không chỉ xem xét giá trị và gradient của hàm mục tiêu mà còn xem xét độ cong của nó, có thể giúp ích trong trường hợp này. Mặc dù các phương pháp này không thể áp dụng trực tiếp cho học sâu do chi phí tính toán, chúng cung cấp trực giác hữu ích để thiết kế các thuật toán tối ưu hóa nâng cao, mô phỏng nhiều đặc tính mong muốn của các thuật toán được trình bày dưới đây.
### Phương Pháp Newton
Xem lại khai triển Taylor của hàm $f: \mathbb{R}^d \rightarrow \mathbb{R}$, không cần dừng lại sau hạng đầu tiên. Thực tế, ta có thể viết:
$$f(\mathbf{x}+\boldsymbol{\epsilon})=f(\mathbf{x})+\boldsymbol{\epsilon}^{\top} \nabla f(\mathbf{x})+\frac{1}{2} \boldsymbol{\epsilon}^{\top} \nabla^2 f(\mathbf{x}) \boldsymbol{\epsilon}+O\left(|\boldsymbol{\epsilon}|^3\right).$$
Để tránh ký hiệu phức tạp, ta định nghĩa $\mathbf{H} \stackrel{\text{def}}{=} \nabla^2 f(\mathbf{x})$ là ma trận Hessian của $f$, một ma trận $d \times d$. Với $d$ nhỏ và bài toán đơn giản, $\mathbf{H}$ dễ tính toán. Tuy nhiên, với mạng nơ-ron sâu, $\mathbf{H}$ có thể quá lớn để lưu trữ, do chi phí lưu trữ $O\left(d^2\right)$ phần tử. Hơn nữa, việc tính toán qua lan truyền ngược có thể quá tốn kém. Hiện tại, hãy bỏ qua các cân nhắc này và xem xét thuật toán ta sẽ nhận được.
Sau cùng, cực tiểu của $f$ thỏa mãn $\nabla f=0$. Theo các quy tắc vi tích phân trong Phần 2.4.3, bằng cách lấy đạo hàm của (12.3.8) theo $\epsilon$ và bỏ qua các hạng bậc cao, ta được:
$$\nabla f(\mathbf{x})+\mathbf{H} \boldsymbol{\epsilon}=0 \text{ và do đó } \boldsymbol{\epsilon}=-\mathbf{H}^{-1} \nabla f(\mathbf{x}).$$
Tức là, ta cần nghịch đảo ma trận Hessian $\mathbf{H}$ như một phần của bài toán tối ưu hóa.
Ví dụ đơn giản, với $f(x)=\frac{1}{2}x^2$, ta có $\nabla f(x)=x$ và $\mathbf{H}=1$. Do đó, với mọi $x$, ta được $\epsilon=-x$. Nói cách khác, chỉ một bước là đủ để hội tụ hoàn hảo mà không cần bất kỳ điều chỉnh nào! Ở đây, ta hơi may mắn: khai triển Taylor chính xác vì $f(x+\epsilon)=\frac{1}{2}x^2+\epsilon x+\frac{1}{2}\epsilon^2$.
Hãy xem điều gì xảy ra trong các bài toán khác. Với hàm hyperbolic cosine lồi $f(x)=\cosh(cx)$ với một hằng số $c$, ta thấy cực tiểu toàn cục tại $x=0$ được đạt sau vài lần lặp.

In [ ]:
c = torch.tensor(0.5)

def f(x):  # objective function
    return torch.cosh(c * x)

def f_grad(x):  # Gradient of the objective function
    return c * torch.sinh(c * x)

def f_hess(x):  # Hessian of the objective function
    return c ** 2 * torch.cosh(c * x)

def newton(eta=1):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * f_grad(x) / f_hess(x)
        results.append(float(x))
    print('epoch 10, x:', x)
    return results

show_trace(newton(), f)

epoch 10, x: tensor(0.)

Bây giờ, hãy xem xét một hàm không lồi, chẳng hạn $f(x)=x \cos(cx)$ với một hằng số $c$. Lưu ý rằng trong phương pháp Newton, ta chia cho Hessian. Điều này có nghĩa là nếu đạo hàm bậc hai âm, ta có thể đi theo hướng làm tăng giá trị của $f$. Đó là một lỗ hổng nghiêm trọng của thuật toán. Hãy xem điều gì xảy ra trong thực tế.

In [ ]:
c = torch.tensor(0.15 * np.pi)

def f(x):  # Objective function
    return x * torch.cos(c * x)

def f_grad(x):  # Gradient of the objective function
    return torch.cos(c * x) - c * x * torch.sin(c * x)

def f_hess(x):  # Hessian of the objective function
    return -2 * c * torch.sin(c * x) - c ** 2 * x * torch.cos(c * x)

show_trace(newton(), f)

In [ ]:
epoch 10, x: tensor(26.8341)

Điều này đã sai lầm nghiêm trọng. Làm thế nào để sửa nó? Một cách là "sửa" Hessian bằng cách lấy giá trị tuyệt đối của nó. Một chiến lược khác là đưa lại tốc độ học.
Với tốc độ học nhỏ hơn một chút, chẳng hạn $\eta=0.5$, ta thấy thuật toán hoạt động hiệu quả hơn.

In [ ]:
show_trace(newton(0.5), f)

epoch 10, x: tensor(7.2699)

### Phân Tích Hội Tụ
Ta chỉ phân tích tốc độ hội tụ của phương pháp Newton cho một hàm mục tiêu lồi và khả vi ba lần, trong đó đạo hàm bậc hai khác không, tức là $f^{\prime\prime}>0$. Bằng chứng đa biến là phần mở rộng trực tiếp của lập luận một chiều dưới đây và được bỏ qua vì nó không giúp nhiều về mặt trực giác.
Gọi $x^{(k)}$ là giá trị của $x$ tại lần lặp thứ $k$ và đặt $e^{(k)} \stackrel{\text{def}}{=} x^{(k)}-x^*$ là khoảng cách từ điểm tối ưu tại lần lặp thứ $k$. Bằng khai triển Taylor, ta có điều kiện $f^{\prime}\left(x^{(*)}\right)=0$ có thể được viết là:
$$0=f^{\prime}\left(x^{(k)}-e^{(k)}\right)=f^{\prime}\left(x^{(k)}\right)-e^{(k)} f^{\prime\prime}\left(x^{(k)}\right)+\frac{1}{2}\left(e^{(k)}\right)^2 f^{\prime\prime\prime}\left(\xi^{(k)}\right),$$
điều này đúng với một $\xi^{(k)} \in \left[x^{(k)}-e^{(k)}, x^{(k)}\right]$. Chia khai triển trên cho $f^{\prime\prime}\left(x^{(k)}\right)$, ta được:
$$e^{(k)}-\frac{f^{\prime}\left(x^{(k)}\right)}{f^{\prime\prime}\left(x^{(k)}\right)}=\frac{1}{2}\left(e^{(k)}\right)^2 \frac{f^{\prime\prime\prime}\left(\xi^{(k)}\right)}{f^{\prime\prime}\left(x^{(k)}\right)} .$$
Nhớ rằng ta có cập nhật $x^{(k+1)}=x^{(k)}-f^{\prime}\left(x^{(k)}\right) / f^{\prime\prime}\left(x^{(k)}\right)$. Thay vào phương trình cập nhật này và lấy giá trị tuyệt đối của cả hai vế, ta có:
$$\left|e^{(k+1)}\right|=\frac{1}{2}\left(e^{(k)}\right)^2 \frac{\left|f^{\prime\prime\prime}\left(\xi^{(k)}\right)\right|}{f^{\prime\prime}\left(x^{(k)}\right)} .$$
Do đó, bất cứ khi nào ta ở trong một vùng có $\left|f^{\prime\prime\prime}\left(\xi^{(k)}\right)\right| /\left(2 f^{\prime\prime}\left(x^{(k)}\right)\right) \leq c$, ta có sai số giảm bậc hai:
$$\left|e^{(k+1)}\right| \leq c\left(e^{(k)}\right)^2 .$$
Lưu ý rằng các nhà nghiên cứu tối ưu hóa gọi đây là hội tụ tuyến tính, trong khi một điều kiện như $\left|e^{(k+1)}\right| \leq \alpha\left|e^{(k)}\right|$ được gọi là tốc độ hội tụ hằng số. Phân tích này đi kèm với một số lưu ý. Thứ nhất, ta không thực sự có đảm bảo khi nào sẽ đạt được vùng hội tụ nhanh. Thay vào đó, ta chỉ biết rằng một khi đạt được, hội tụ sẽ rất nhanh. Thứ hai, phân tích này yêu cầu $f$ có tính chất tốt đến các đạo hàm bậc cao. Nó phụ thuộc vào việc đảm bảo rằng $f$ không có bất kỳ đặc tính "bất ngờ" nào về cách nó có thể thay đổi giá trị.

### Tiền Điều Kiện Hóa
Không ngạc nhiên khi việc tính toán và lưu trữ toàn bộ Hessian rất tốn kém. Do đó, việc tìm các giải pháp thay thế là mong muốn. Một cách để cải thiện là tiền điều kiện hóa. Nó tránh tính toán toàn bộ Hessian mà chỉ tính các phần tử đường chéo. Điều này dẫn đến các thuật toán cập nhật dạng:
$$\mathbf{x} \leftarrow \mathbf{x}-\eta \operatorname{diag}(\mathbf{H})^{-1} \nabla f(\mathbf{x}) .$$
Mặc dù điều này không tốt bằng phương pháp Newton đầy đủ, nó vẫn tốt hơn nhiều so với việc không sử dụng. Để thấy tại sao đây là ý tưởng tốt, hãy xem xét một tình huống mà một biến biểu thị chiều cao tính bằng milimet và một biến khác biểu thị chiều cao tính bằng kilômét. Giả sử rằng với cả hai, tỷ lệ tự nhiên là mét, ta có sự không khớp lớn trong tham số hóa. May mắn thay, việc sử dụng tiền điều kiện hóa sẽ loại bỏ điều này. Hiệu quả, tiền điều kiện hóa với hạ gradient tương đương với việc chọn một tốc độ học khác nhau cho mỗi biến (tọa độ của vector $\mathbf{x}$). Như ta sẽ thấy sau, tiền điều kiện hóa thúc đẩy một số cải tiến trong các thuật toán tối ưu hóa hạ gradient ngẫu nhiên.
### Hạ Gradient với Tìm Kiếm Tuyến
Một trong những vấn đề chính trong hạ gradient là ta có thể vượt quá mục tiêu hoặc tiến bộ không đủ. Một cách sửa đơn giản là sử dụng tìm kiếm tuyến kết hợp với hạ gradient. Tức là, ta sử dụng hướng được cho bởi $\nabla f(\mathbf{x})$ và sau đó thực hiện tìm kiếm nhị phân để xác định tốc độ học $\eta$ nào tối ưu hóa $f(\mathbf{x}-\eta \nabla f(\mathbf{x}))$.
Thuật toán này hội tụ nhanh chóng (xem phân tích và chứng minh, ví dụ, Boyd và Vandenberghe (2004)). Tuy nhiên, đối với mục đích học sâu, điều này không thực sự khả thi, vì mỗi bước của tìm kiếm tuyến sẽ yêu cầu đánh giá hàm mục tiêu trên toàn bộ tập dữ liệu. Điều này quá tốn kém để thực hiện.

## Excercises